CS 178 Project <br>
Team 69 - "Adulting" <br>
Members:<br>
Dingyi Chang #89167697<br>
Nicholas King #33099364<br>
Jenny My Lee #<br>

We will first download the data (the Adult dataset from UCI ML Repo) as well as importing any necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

with requests.get(url) as link:
    adult = 